# Colab Setting

## Google Drive Mount

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

## Change Root Directory

In [2]:
# import os
# path = '/content/drive/MyDrive/Storage/Github/hyuckjinkim/data-scientist-competitions/Dacon/23_퍼즐이미지맞추기/'
# os.chdir(path)

<br></br>

# Setting

## Install

In [3]:
# !pip install segmentation-models-pytorch

## Import

In [4]:
import random
import pandas as pd
import numpy as np
import os
from copy import deepcopy
import time
import gc

from PIL import Image
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
import torchvision.models as models

import segmentation_models_pytorch as smp

from tqdm.auto import tqdm, trange

import warnings
warnings.filterwarnings(action='ignore')

/Users/khj/anaconda3/envs/torch_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import os
def mkdir(paths,verbose=True)->None:
    if isinstance(paths,str):
        paths = [paths]
    for path in paths:
        if not os.path.exists(path):
            os.mkdir(path)
            if verbose:
                print('directory created: {}'.format(path))

In [6]:
def flush():
    while True:
        k = gc.collect()
        if k==0:
            break
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

In [7]:
def seed_everything(seed=42):
    '''
    Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.
    '''
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)

## Config Setting

In [8]:
class CFG:
    SEED = 42
    IMG_SIZE = 224
    EPOCHS = 10
    LR = 1e-3
    BATCH_SIZE = 64
    DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [9]:
seed_everything(CFG.SEED)

<br></br>

# Data Prepare

## Data Load

In [10]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

train_df['img_path'] = train_df['img_path'].apply(lambda x: x.replace('./','./data/'))
test_df ['img_path'] = test_df ['img_path'].apply(lambda x: x.replace('./','./data/'))

In [11]:
# train_df = train_df[:1000]

## Train Validation Split

In [12]:
from sklearn.model_selection import train_test_split

tr_df, val_df = train_test_split(train_df,test_size=0.3,shuffle=True,random_state=CFG.SEED)

In [13]:
tr_labels  = tr_df .iloc[:,2:].values.reshape(-1, 4, 4)
val_labels = val_df.iloc[:,2:].values.reshape(-1, 4, 4)

In [14]:
(tr_labels.shape, val_labels.shape)

((49000, 4, 4), (21000, 4, 4))

## Data Loader

In [16]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transform=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transform = transform

    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform is not None:
            image = self.transform(image)

        if self.label_list is not None:
            label = torch.tensor(self.label_list[index], dtype=torch.long) - 1
            return image, label
        else:
            return image

    def __len__(self):
        return len(self.img_path_list)

In [17]:
# from joblib import Parallel, delayed

# class CustomDataset(Dataset):
#     def __init__(self, img_paths, labels, transform=None, n_jobs=os.cpu_count()):
#         self.labels = labels
#         self.transform = transform

#         self.images = Parallel(n_jobs=n_jobs)(
#             delayed(self._get_image)(img_path) for img_path in tqdm(img_paths)
#         )
        
#         if self.labels is not None:
#             self.labels = []
#             for label in labels:
#                 label = torch.tensor(label, dtype=torch.long) - 1
#                 self.labels.append(label)

#     def _get_image(self,img_path):
#         image = cv2.imread(img_path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         if self.transform is not None:
#             image = self.transform(image)
#         return image

#     def __getitem__(self, index):
#         if self.labels is not None:
#             return self.images[index], self.labels[index]
#         else:
#             return self.images[index]

#     def __len__(self):
#         return len(self.images)

In [18]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((CFG.IMG_SIZE, CFG.IMG_SIZE)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((CFG.IMG_SIZE, CFG.IMG_SIZE)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [19]:
train_dataset = CustomDataset(tr_df['img_path'].values, tr_labels, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG.BATCH_SIZE, shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_df['img_path'].values, val_labels, test_transform)
val_loader = DataLoader(val_dataset, batch_size = CFG.BATCH_SIZE, shuffle=False, num_workers=0)

test_dataset = CustomDataset(test_df['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG.BATCH_SIZE, shuffle=False, num_workers=0)

<br></br>

# Modeling

## Model Define

In [20]:
class PuzzleModel(nn.Module):
    def __init__(self):
        super(PuzzleModel, self).__init__()
        self.model = smp.Unet(encoder_name="resnet34", encoder_weights="imagenet")

        self.final_pool = nn.MaxPool2d(2)
        self.final_conv = nn.Conv2d(1, 16, kernel_size=28, stride=28)
        self.final_bn = nn.BatchNorm2d(16)

    def forward(self, x):
        x = self.model(x)
        x = self.final_pool(x)
        x = self.final_conv(x)
        x = self.final_bn(x) # (B,16,4,4)
        return x

In [21]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class ConvBlock(nn.Module):
#     def __init__(self, in_channels, out_channels, kernel_size=3):
#         super(ConvBlock, self).__init__()
#         self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, padding=1)
#         self.bn = nn.BatchNorm2d(out_channels)
#         self.relu = nn.ReLU()

#     def forward(self, x):
#         return self.relu(self.bn(self.conv(x)))

# class DeconvBlock(nn.Module):
#     def __init__(self, in_channels, mid_channels, out_channels):
#         super(DeconvBlock, self).__init__()
#         self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
#         self.conv_mid = nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1)
#         self.bn_mid = nn.BatchNorm2d(mid_channels)
#         self.relu_mid = nn.ReLU()
#         self.conv_block = ConvBlock(mid_channels + mid_channels, out_channels)

#     def forward(self, x, skip):
#         x = self.up(x)
#         x = self.relu_mid(self.bn_mid(self.conv_mid(x)))
#         x = torch.cat((x, skip), dim=1)
#         return self.conv_block(x)

# class BaseModel(nn.Module):
#     def __init__(self):
#         super(BaseModel, self).__init__()
#         # Contraction path
#         self.conv1 = ConvBlock(3, 16)
#         self.pool1 = nn.MaxPool2d(2)
#         self.conv2 = ConvBlock(16, 32)
#         self.pool2 = nn.MaxPool2d(2)
#         self.conv3 = ConvBlock(32, 64)
#         self.pool3 = nn.MaxPool2d(2)
#         self.conv4 = ConvBlock(64, 128)
#         self.pool4 = nn.MaxPool2d(2)
#         self.conv5 = ConvBlock(128, 256)

#         # Expansion path
#         self.up6 = DeconvBlock(256, 128, 128)
#         self.up7 = DeconvBlock(128, 64, 64)
#         self.up8 = DeconvBlock(64, 32, 32)
#         self.up9 = DeconvBlock(32, 16, 16)

#         self.final_pool = nn.MaxPool2d(2)
#         self.final_conv = nn.Conv2d(16, 16, kernel_size=28, stride=28)
#         self.final_bn = nn.BatchNorm2d(16)

#     def forward(self, x):
#         # Contraction path
#         x1 = self.conv1(x)
#         x = self.pool1(x1)
#         x2 = self.conv2(x)
#         x = self.pool2(x2)
#         x3 = self.conv3(x)
#         x = self.pool3(x3)
#         x4 = self.conv4(x)
#         x = self.pool4(x4)
#         x5 = self.conv5(x)

#         # Expansion path
#         x = self.up6(x5, x4)
#         x = self.up7(x, x3)
#         x = self.up8(x, x2)
#         x = self.up9(x, x1)

#         x = self.final_pool(x)
#         out = self.final_bn(self.final_conv(x)) # (B,16,4,4)
#         return out

## Train Validation Define

In [22]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path:str=None, trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: None
            trace_func (function): trace print function.
                            Default: print
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        # if self.verbose:
        #     self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        if self.path is not None:
            torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [23]:
def train(
    model, train_loader, valid_loader, criterion, epochs,
    optimizer, scheduler=None, early_stopping=None, device='cpu', metric_period=1,
    verbose=True, save_dir = './checkpoints',
):
    global img, label, output

    model.to(device)
    criterion.to(device)

    best_loss  = 999999999
    best_epoch = 1
    best_model = None
    is_best    = np.nan
    best_model_state = deepcopy(model.state_dict())

    start_time = time.time()
    epoch_s = time.time()
    for epoch in range(1, epochs+1):

        # training step
        train_loss_list = []
        train_acc_list = []
        model.train()
        for img, label in tqdm(iter(train_loader),desc='Train Batch'):
            img = img.to(device)
            label = label.to(device)

            optimizer.zero_grad()
            output = model(img)#.float()

            loss = criterion(output, label)

            loss.backward()  # Getting gradients
            optimizer.step() # Updating parameters

            train_loss_list.append(loss.item())

            predicted_label = torch.argmax(output, dim=1)
            for prediction, actual in zip(predicted_label, label):
                acc = ((prediction == actual).sum() / 16).item()
                train_acc_list.append(acc)

        train_loss = np.mean(train_loss_list)
        train_acc  = np.mean(train_acc_list)

        # valiation step
        valid_loss_list = []
        valid_acc_list = []
        model.eval()
        with torch.no_grad():
            for img, label in tqdm(iter(valid_loader),desc='Validation'):
                img = img.to(device)
                label = label.to(device)
                output = model(img)#.float()

                loss = criterion(output, label)

                valid_loss_list.append(loss.item())

                predicted_label = torch.argmax(output, dim=1)
                for prediction, actual in zip(predicted_label, label):
                    acc = ((prediction == actual).sum() / 16).item()
                    valid_acc_list.append(acc)

            valid_loss = np.mean(valid_loss_list)
            valid_acc  = np.mean(valid_acc_list)

        epoch_e = time.time()

        if scheduler is not None:
            scheduler.step(valid_loss)

        # update the best epoch & best loss
        if (best_loss > valid_loss) | (epoch==1):
            best_epoch = epoch
            best_loss = valid_loss
            best_model = model
            is_best = 1
            best_model_state = deepcopy(model.state_dict())
            save_path = os.path.join(save_dir,'epoch({})-val_loss({:.3f}).pt'.format(epoch,valid_loss))
            torch.save(best_model_state, save_path)
        else:
            is_best = 0

        # 결과물 printing
        if (verbose) & (epoch % metric_period == 0):
            mark = '*' if is_best else ' '
            epoch_str = str(epoch).zfill(len(str(epochs)))
            progress = '{}[{}/{}] loss: {:.4f}, acc: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}, best_epoch: {}, elapsed: {:.2f}s, total: {:.2f}s, remaining: {:.2f}s'\
                .format(
                    mark,
                    epoch_str,
                    epochs,
                    train_loss,
                    train_acc,
                    valid_loss,
                    valid_acc,
                    best_epoch,
                    epoch_e-epoch_s,
                    epoch_e-start_time,
                    (epoch_e-epoch_s)*(epochs-epoch)/metric_period,
                )
            epoch_s = time.time()
            print(progress)

        # early stopping 여부를 체크. 현재 과적합 상황 추적
        if early_stopping is not None:
            early_stopping(valid_loss, model)
            if early_stopping.early_stop:
                break

        #print(model.state_dict()['linear.weight'])

    model.load_state_dict(best_model_state)
    return best_model

In [24]:
# def train(model, optimizer, train_loader, val_loader, device):
#     model.to(device)
#     criterion = nn.CrossEntropyLoss().to(device)

#     best_val_acc = 0
#     best_model = None
#     for epoch in range(1, CFG['EPOCHS']+1):
#         model.train()
#         train_loss = []
#         for imgs, labels in tqdm(iter(train_loader),desc='train batch'):
#             imgs = imgs.float().to(device)
#             labels = labels.to(device)

#             optimizer.zero_grad()

#             output = model(imgs)
#             loss = criterion(output, labels)

#             loss.backward()
#             optimizer.step()

#             train_loss.append(loss.item())

#         _val_loss, _val_acc = validation(model, criterion, val_loader, device)
#         _train_loss = np.mean(train_loss)
#         print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val ACC : [{_val_acc:.5f}]')

#         if best_val_acc < _val_acc:
#             best_val_acc = _val_acc
#             best_model = model

#     return best_model

# def validation(model, criterion, val_loader, device):
#     model.eval()
#     val_loss = []
#     val_acc = []

#     with torch.no_grad():
#         for imgs, labels in tqdm(iter(val_loader),desc='validation batch'):
#             imgs = imgs.float().to(device)
#             labels = labels.to(device)

#             output = model(imgs)

#             loss = criterion(output, labels)

#             val_loss.append(loss.item())

#             # 정확도 계산을 위한 예측 레이블 추출
#             predicted_labels = torch.argmax(output, dim=1)

#             # 샘플 별 정확도 계산
#             for predicted_label, label in zip(predicted_labels, labels):
#                 val_acc.append(((predicted_label == label).sum() / 16).item())

#         _val_loss = np.mean(val_loss)
#         _val_acc = np.mean(val_acc)

#     return _val_loss, _val_acc

## Fitting

In [25]:
# import glob

# train_img_paths = glob.glob('./data/train/*')
# test_img_paths  = glob.glob('./data/test/*')

# len(train_img_paths), len(test_img_paths)

In [29]:
model = PuzzleModel()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(params = model.parameters(), lr = CFG.LR)

# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
#     optimizer, mode='min', factor=0.5, patience=2, threshold_mode='abs',min_lr=1e-7, verbose=True)
scheduler = None

# early_stopping = EarlyStopping(patience=10, verbose=True, path=None)
early_stopping = None

In [30]:
mkdir('checkpoints')

In [ ]:
flush()

infer_model = train(
    model,
    train_loader, val_loader,
    criterion,
    CFG.EPOCHS,
    optimizer, scheduler, early_stopping,
    CFG.DEVICE,
    metric_period = 1,
    verbose = True,
    save_dir = 'checkpoints',
)

Train Batch:  30%|███████▏                | 230/766 [1:03:51<2:27:16, 16.49s/it]

<br></br>

# Inference

## Define Inference

In [ ]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)

            output = model(imgs)

            # 정확도 계산을 위한 예측 레이블 추출
            predicted_labels = torch.argmax(output, dim=1).view(-1, 16)
            predicted_labels = predicted_labels.cpu().detach().numpy()

            preds.extend(predicted_labels)

    return preds

In [ ]:
preds = inference(infer_model, test_loader, device)

## Submission

In [ ]:
submit = pd.read_csv('./sample_submission.csv')

In [ ]:
submit.iloc[:, 1:] = preds
submit.iloc[:, 1:] += 1

In [ ]:
submit.to_csv('./baseline_submit.csv', index=False)